### 1.  Installations and Settings 🛠️

In [17]:
%%bash
uv pip install -q llama-index-core
uv pip install -q llama-index-llms-groq
uv pip install -q llama-index-readers-file
uv pip install -q llama-index-embeddings-huggingface
uv pip install -q llama-index-embeddings-instructor

In [18]:
import os

In [19]:
from google.colab import userdata

# Set the token as an environ variable
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["sina_hug_tocken"] = userdata.get("sina_hug_tocken")

In [20]:
from llama_index.llms.groq import Groq

# This info's at the top of each HuggingFace model page
model = "llama3-70b-8192"

llm = Groq(
    model=model)

Now our model need some information to work.

In [21]:
%mkdir -p /content/data
!wget -O /content/data/The_Republic_of_Plato.txt https://www.gutenberg.org/cache/epub/55201/pg55201.txt

--2025-05-06 15:20:40--  https://www.gutenberg.org/cache/epub/55201/pg55201.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1457578 (1.4M) [text/plain]
Saving to: ‘/content/data/The_Republic_of_Plato.txt’

/content/data/The_R 100%[===================>]   1.39M  2.39MB/s    in 0.6s    

2025-05-06 15:20:43 (2.39 MB/s) - ‘/content/data/The_Republic_of_Plato.txt’ saved [1457578/1457578]



In [22]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("/content/data").load_data()

Large documents, like books, are too big for models to handle all at once. To make them easier to process, we split them into smaller parts, called chunks. These chunks can be split:

By `sentences` – keeping full sentences together.


By `tokens` – based on model input limits (tokens = pieces of words).


By `meaning` (semantics) – keeping related ideas grouped.

In [23]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=800, chunk_overlap=150)

docs = text_splitter.get_nodes_from_documents(documents)

**Creating vectors with embeddings**

Embeddings are a fancy way of saying we turn words into numbers that computers can understand. Each word gets its own unique code, based on its meaning and relationship to other words. The list of numbers produced is known as a vector. Vectors allow us to compare text and find chunks that contain similar information

In [24]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding


embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_folder = "/content/embedding_model/"


embeddings = HuggingFaceEmbedding(
                                   model_name = embedding_model,
                                   cache_folder = embeddings_folder)

In [25]:
test_text = "Sina is my friend"
query_result = embeddings.get_text_embedding(test_text)
query_result

[-0.08157619088888168,
 0.04829923063516617,
 -0.09897464513778687,
 -0.017182614654302597,
 -0.03681368753314018,
 0.03711341693997383,
 0.07499756664037704,
 -0.028513623401522636,
 0.04212306812405586,
 0.006139792501926422,
 0.01407662034034729,
 -0.06968077272176743,
 -0.06041228026151657,
 0.06358020007610321,
 0.06774589419364929,
 -0.033906325697898865,
 0.07246381789445877,
 -0.003503041807562113,
 -0.034613173454999924,
 -0.06648339331150055,
 -0.04274279996752739,
 0.03157821670174599,
 0.008334643207490444,
 -0.027281727641820908,
 -0.08411352336406708,
 0.05708814784884453,
 0.056152693927288055,
 -0.0463726781308651,
 -0.03786627948284149,
 -0.102239228785038,
 -0.025849023833870888,
 0.12577001750469208,
 -0.06357301026582718,
 -0.05562366172671318,
 -0.03052680753171444,
 0.06544873863458633,
 -0.030589813366532326,
 -0.04039536789059639,
 -0.0035911700688302517,
 0.05368359386920929,
 -0.029433434829115868,
 0.07776978611946106,
 0.01670188643038273,
 -0.06749193370342

In [26]:
characters = len(test_text)
dimensions = len(query_result)
print(
    f"The {characters} character sentence was transformed into a {dimensions} dimension vector"
)

The 17 character sentence was transformed into a 384 dimension vector


**Creating a vector database**

Imagine a library where books aren't just filed alphabetically, but also by their themes, characters, and emotions. That's the magic of vector databases: they unlock information beyond keywords, connecting ideas in unexpected ways.

In [27]:
from llama_index.core import VectorStoreIndex

documents = SimpleDirectoryReader("/content/data").load_data()

vector_index = VectorStoreIndex.from_documents(
                                                documents,
                                                transformations = [text_splitter],
                                                embed_model = embeddings)

Now, I want to save the database

In [28]:
vector_index.storage_context.persist(persist_dir="/content/vector_index")

In [29]:
# code below is for future, when we want to load the database:

# from llama_index.core import StorageContext, load_index_from_storage

# storage_context = StorageContext.from_defaults(persist_dir="/content/vector_index")
# vector_index = load_index_from_storage(storage_context, embed_model=embeddings)

**Adding a prompt**

We can guide our model's behavior with a prompt.

In [30]:
from llama_index.core.prompts import PromptTemplate

input_template = """Here is the context:
{context_str}

Answer the question based only on the following context. Keep your answers short and succinct.
Question to be answered: {query_str}
Answer:"""

prompt = PromptTemplate(template=input_template)

**RAG - chaining it all together**

This is the final piece of the puzzle:

we now drive everything with an engine. Our vector database, our prompt, and our LLM join to give us retrieval augmented generation

In [31]:
query_engine = vector_index.as_query_engine(
                                            llm=llm,
                                            text_qa_template=prompt,
                                            similarity_top_k=2,
                                            response_mode="tree_summarize")

Let's test our model

In [32]:
answer = await query_engine.aquery("Who is Plato?")
print(answer)

Plato is a philosopher who has deeply meditated on the 'way of life of Pythagoras' and his followers, and has been influenced by Pythagoreanism in his philosophy and the form of his State.
